In [162]:
import pickle 
import gensim
import os
import sys
import json
import numpy as np
from acrlist import acr
import tensorflow as tf
from keras.models import Sequential
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from keras.layers import *
from keras.optimizers import *
from keras.models import load_model

In [163]:
train_data_tweets = []
train_data_acronyms = []
test_data_tweets = []
test_data_acronyms = []

for expansion in acr['lol']:
    with open("train_data/"+str(expansion)+".txt") as file:
      tweets = file.readlines()
      for tweet in tweets:
        tweet = json.loads(tweet)
        tweet = tweet.lower()
        tweet = tweet.replace(expansion, 'lol')
        train_data_tweets.append(tweet.split())
        train_data_acronyms.append(expansion)
    with open("test_data/"+str(expansion)+".txt") as file:
      tweets = file.readlines()
      for tweet in tweets:
        tweet = json.loads(tweet)
        tweet = tweet.lower()
        tweet = tweet.replace(expansion, 'lol')
        test_data_tweets.append(tweet.split())
        test_data_acronyms.append(expansion)

In [164]:
def create_tagged_document(split_tweets, data_acronyms):
  for i, tweet in enumerate(split_tweets):
    yield gensim.models.doc2vec.TaggedDocument(words=tweet, tags=[data_acronyms[i]])
    
train_data = list(create_tagged_document(train_data_tweets, train_data_acronyms))
test_data = list(create_tagged_document(test_data_tweets, test_data_acronyms))
print(train_data[1])
print(train_data[0])
print(test_data[1])
print(test_data[0])

TaggedDocument(['lilly', 'singh’s', 'desi', 'spoof', 'of', '‘game', 'of', 'thrones’', 'has', 'left', 'indian', 'fans', 'lol', '-', 'the', 'indian', 'express', 'https://t.co/vuwtu6xnmv'], ['laughing out loud'])
TaggedDocument(['@latah8as', '@treyancy3', 'i’m', 'actually', 'lol', '😂'], ['laughing out loud'])
TaggedDocument(['@pamela_alexisss', 'i’m', 'lol', 'lmao'], ['laughing out loud'])
TaggedDocument(['i', 'just', 'had', 'to', 'lol', 'at', 'this', 'photo', '@aoc', 'and', '@speakerpelosi', 'very', 'funny', 'stuff!!', 'still', 'lol..'], ['laughing out loud'])


In [167]:
dmodel = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=50)
dmodel.build_vocab(train_data)
dmodel.train(train_data, total_examples=dmodel.corpus_count, epochs=dmodel.epochs)

In [168]:
v = dmodel.infer_vector("i wanna buy some skins in lol".split())
res_tup = dmodel.docvecs.most_similar([v])[0]
print("Doc2Vec Vector & Prediction:")
print(np.array(v))
print(res_tup)
print(train_data[0].tags[0])
print(train_data[0].words)
print(dmodel.infer_vector(train_data[0].words))
print(dmodel.infer_vector(train_data[0].words, steps=20))

Doc2Vec Vector & Prediction:
[ 0.53961796 -0.03004197 -0.339505    0.27380317  0.06550015  0.29495034
 -0.03175234  0.05654053 -0.16397199 -0.25769436 -0.16722617 -0.62985784
 -0.02210142 -0.10580397  0.2434161  -0.44827366  0.33367106 -0.2998599
 -0.05859925  0.17700538  0.42669865  0.08026007  0.10830867  0.2661799
  0.00607733  0.33039805 -0.09610986  0.54567415 -0.11120132  0.2787963
  0.09548727  0.1044259   0.22753464 -0.04366215 -0.33720726 -0.15046257
 -0.37890062 -0.26515806 -0.14796767  0.24082665 -0.35714954 -0.7082849
 -0.02188265  0.23272677 -0.41123557 -0.27309862 -0.6141171   0.1518365
  0.6649613   0.07343742]
('league of legends', 0.4654266834259033)
laughing out loud
['@latah8as', '@treyancy3', 'i’m', 'actually', 'lol', '😂']
[ 0.23981173 -0.01939626 -0.07747637 -0.16330108 -0.14097245 -0.05572887
 -0.20805207 -0.1392624   0.17028673 -0.04991067  0.24753767  0.10406754
  0.20364624  0.13190307 -0.03903967  0.15691389  0.20277162 -0.14576311
 -0.07028452  0.11997699  0.

In [119]:
# def classifier(X_train, Y_train, X_test, Y_test):
#   #takes Doc2Vec as input layer instead of Word Embeddings, and trains classifiers for each acronym
#   tf_model = Sequential()
# #   tf_model.add(Flatten(input_shape=(1, 50)))
# #   tf_model.add(Dense(128, activation="relu"))
# #   tf_model.add(Dense(64, activation="relu"))
# #   tf_model.add(Dense(2, activation="softmax"))
#   tf_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])
#   tf_model.fit(X_train, Y_train, epochs=3)
#   score, acc = tf_model.evaluate(X_test, Y_test, verbose=1, batch_size=32)
#   print("Score: %.2f" % (score))
#   print("Validation Accuracy: %.2f" % (acc))
#   return tf_model

In [169]:
X_train = []
X_test = []
Y_train = []
Y_test = []
CATEGORIES = ['laughing out loud', 'league of legends', 'lots of love', 'little old lady']
def vector_for_learning(model, input_docs):
    sents = input_docs
    targets, feature_vectors = zip(*[(CATEGORIES.index(doc.tags[0]), np.array(model.infer_vector(doc.words))) for doc in sents])
    return targets, feature_vectors
'''
for expansion in acr['gg']:
    with open("train_data/"+str(expansion)+".txt") as file:
      tweets = file.readlines()
      for tweet in tweets:
        tweet = json.loads(tweet)
        tweet = tweet.lower()
        tweet = tweet.replace(expansion, 'gg')
        X_train.append(np.array(model.infer_vector(tweet.split())))
        Y_train.append(expansion)
    with open("test_data/"+str(expansion)+".txt") as file:
      tweets = file.readlines()
      for tweet in tweets:""
        tweet = json.loads(tweet)
        tweet = tweet.lower()
        tweet = tweet.replace(expansion, 'gg')
        X_test.append(np.array(model.infer_vector(tweet.split())))
        Y_test.append(expansion)
'''
Y_train, X_train = vector_for_learning(dmodel,train_data)
Y_test, X_test = vector_for_learning(dmodel,test_data)
print(X_train[0])
print(Y_train[0])
# Y_train = np.asarray(Y_train)

[ 2.72146732e-01  1.27608376e-02  7.19396099e-02 -7.57892132e-02
 -1.84000164e-01  5.43659786e-03 -2.13307098e-01 -8.28714818e-02
  2.26879060e-01 -9.19091403e-02  2.83817112e-01  1.01330645e-01
  1.75084636e-01  1.60453498e-01  1.52820811e-01  6.35252371e-02
  1.80931166e-01 -9.19022486e-02 -1.13736913e-01  1.82115555e-01
  4.11402375e-01  3.66346031e-01  1.30262151e-01 -2.94576604e-02
  7.39453584e-02 -2.63283879e-01  1.13482416e-01 -7.00486302e-02
 -1.26692966e-01 -1.22657302e-03 -3.56108874e-01 -3.91679481e-02
 -2.34017111e-02 -9.78079885e-02 -1.19065680e-01 -4.88532111e-02
 -1.34767488e-01 -1.62983626e-01 -1.54029101e-01  1.07682928e-01
  3.98038304e-04 -2.13714346e-01 -1.48437560e-01 -1.22529089e-01
 -6.21740669e-02 -2.86420863e-02  1.06267244e-01  2.56128758e-02
  2.67329633e-01 -7.41078481e-02]
0


In [170]:
print(Y_train)

(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [192]:
xx = tf.keras.utils.normalize(X_train)
xtest = tf.keras.utils.normalize(X_test)
Y_train = np.array(Y_train)
model = Sequential()
model.add(Dense(64, input_shape=(50,),activation=tf.nn.relu))
model.add(Dropout(0.3))
model.add(Dense(128, activation=tf.nn.relu))
model.add(Dropout(0.3))
model.add(Dense(128, activation=tf.nn.relu))
model.add(Dropout(0.3))
model.add(Dense(64, activation=tf.nn.relu))
model.add(Dense(4, activation=tf.nn.softmax))
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model.fit(np.array(xx), np.array(Y_train), epochs=50)
val_loss, val_acc = model.evaluate(np.array(xtest), np.array(Y_test))
print(val_loss, val_acc)

Epoch 1/50
16495/16495 [==============================] - 3s 187us/step - loss: 0.5800 - acc: 0.7917
Epoch 2/50
16495/16495 [==============================] - 1s 72us/step - loss: 0.3910 - acc: 0.8609
Epoch 3/50
16495/16495 [==============================] - 1s 70us/step - loss: 0.3424 - acc: 0.8801
Epoch 4/50
16495/16495 [==============================] - 1s 69us/step - loss: 0.3207 - acc: 0.8868
Epoch 5/50
16495/16495 [==============================] - 1s 72us/step - loss: 0.3056 - acc: 0.8938
Epoch 6/50
16495/16495 [==============================] - 1s 69us/step - loss: 0.2954 - acc: 0.8946
Epoch 7/50
16495/16495 [==============================] - 1s 68us/step - loss: 0.2875 - acc: 0.8976
Epoch 8/50
16495/16495 [==============================] - 1s 70us/step - loss: 0.2804 - acc: 0.9007
Epoch 9/50
16495/16495 [==============================] - 1s 71us/step - loss: 0.2750 - acc: 0.9018
Epoch 10/50
16495/16495 [==============================] - 1s 70us/step - loss: 0.2711 - acc: 0.903

In [175]:
logreg = LogisticRegression(n_jobs=2, C=1e5)
logreg.fit(X_train, Y_train)
y_pred = logreg.predict(X_test)
print('Testing accuracy for movie plots%s' % accuracy_score(Y_test, y_pred))
print('Testing F1 score for movie plots: {}'.format(f1_score(Y_test, y_pred, average='weighted')))

/home/lol123/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/lol123/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/lol123/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


Testing accuracy for movie plots0.7584517162579216
Testing F1 score for movie plots: 0.7578662722214313
